In [ ]:
import ftplib
import requests

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
# Plot styling.
plt.style.use(['seaborn-white', 'seaborn-paper'])
plt.rc('font', family='serif')
sns.set_palette(['#9e0059', '#6da7de', '#ee266d', '#dee000', '#eb861e'])
sns.set_context('paper', font_scale=1.3)    # Single-column figure.

### Input data

In [ ]:
# Living data analysis performed on 2020-11-18.
living_data_base_url = 'MSV000084314/updates/2020-11-18_mwang87_d115210a/other'
ftp_prefix = f'ftp://massive.ucsd.edu/{living_data_base_url}'
# Get the MassIVE IDs for all datasets included in the living data.
ftp = ftplib.FTP('massive.ucsd.edu')
ftp.login()
ftp.cwd(f'{living_data_base_url}/CLUSTERINFO')
msv_ids = [filename[:filename.find('_')] for filename in ftp.nlst()]

In [ ]:
# Collect the number of MS2 spectra per dataset and file.
# The number of MS2 spectra per file was determined using grep:
# while read filename; do if [[ "$filename" == *.mz[Xx][Mm][Ll] ]]; then echo $filename:$(grep -c "msLevel=\"2\"" "/data/massive/$filename"); elif [[ "$filename" == *.mz[Mm][Ll] ]]; then echo $filename:$(grep -c "name=\"ms level\" value=\"2\"" /data/massive/"$filename"); fi done < filenames.txt > filenames_ms2.txt
filenames = pd.DataFrame(
    pd.read_csv('filenames_ms2.txt', header=None, names=['filename'],
                squeeze=True)
    .str.rsplit(':', 1).to_list(), columns=['filename', 'spectra_ms2'])
filenames['dataset'] = filenames['filename'].str.split('/', 1).str[0]
filenames['spectra_ms2'] = filenames['spectra_ms2'].astype(int)
# Add dataset titles.
titles = requests.get('https://massive.ucsd.edu/ProteoSAFe/datasets_json.jsp'
                      '#%7B%22query%22%3A%7B%7D%2C%22table_sort_history'
                      '%22%3A%22createdMillis_dsc%22%7D').json()
filenames = pd.merge(filenames,
                     pd.DataFrame([(ds['dataset'].strip(), ds['title'])
                                   for ds in titles['datasets']],
                                  columns=['dataset', 'title']),
                     'left', on='dataset')

In [ ]:
print(f'Number of datasets: {len(msv_ids)}')
print(f'Number of MS/MS spectra: '
      f'{filenames.loc[filenames["dataset"].isin(msv_ids), "spectra_ms2"].sum():,}')

### Export GNPS library searching tasks

In [ ]:
# Construct a URL to import the datasets into library searching.
datasets, n_datasets_split = filenames['dataset'].unique(), 200
for start_i in range(0, len(datasets), n_datasets_split):
    url = (f'https://gnps.ucsd.edu/ProteoSAFe/index.jsp?params={{'
           f'"workflow":"MOLECULAR-LIBRARYSEARCH-V2",'
           f'"library_on_server":"d.speclibs;",'
           f'"spec_on_server":"'
           f'{";".join("d." + datasets[start_i:start_i + n_datasets_split] + "/ccms_peak")}"}}')
    print(url)

### Evaluate suspect library identification performance

In [ ]:
task_ids_std = ['308b3393', '18cf4e52', 'c0249eb6', 'debd3bbb',
                '8cdb4d7d', 'a9e7e4b1', '334ed0d9', 'b55aef34']
task_ids_sus = ['064be855', 'd243afb8', 'febab54d', 'eba0dfe6',
                'e1d68975', '1df48f2d', 'b7f8c3d4', '50e3d8ae']
filename = '../../data/processed/MOLECULAR-LIBRARYSEARCH-V2-{}-view_all_annotations_DB-main.tsv'
filename_ids_std = (
    pd.concat([pd.read_csv(filename.format(task_id),
                           usecols=['full_CCMS_path'],
                           sep='\t', squeeze=True)
               .value_counts() for task_id in task_ids_std])
    .reset_index()
    .rename(columns={'index': 'filename', 'full_CCMS_path': 'num_ids'}))
filename_ids_sus = (
    pd.concat([pd.read_csv(filename.format(task_id),
                           usecols=['full_CCMS_path'],
                           sep='\t', squeeze=True)
               .value_counts() for task_id in task_ids_sus])
    .reset_index()
    .rename(columns={'index': 'filename', 'full_CCMS_path': 'num_ids'}))

In [ ]:
filename_ids = pd.merge(filename_ids_std, filename_ids_sus, 'outer',
                        'filename', suffixes=['_std', '_sus']).fillna(0)
filename_ids = pd.merge(filenames, filename_ids, 'left', 'filename')
filename_ids = filename_ids.fillna(0)
filename_ids['num_ids_std'] = filename_ids['num_ids_std'].astype(int)
filename_ids['num_ids_sus'] = filename_ids['num_ids_sus'].astype(int)
filename_ids = filename_ids[['dataset', 'filename', 'spectra_ms2',
                             'num_ids_std', 'num_ids_sus']]

In [ ]:
width = 7
height = width / 1.618
fig, ax = plt.subplots(figsize=(width, height))

dataset_ids = (filename_ids.groupby('dataset')
               .agg({'spectra_ms2': 'sum',
                     'num_ids_std': 'sum',
                     'num_ids_sus': 'sum'})
               .reset_index())
dataset_ids['num_ids_std'] /= dataset_ids['spectra_ms2']
dataset_ids['num_ids_sus'] /= dataset_ids['spectra_ms2']
mask = dataset_ids['dataset'].isin(msv_ids)
dataset_ids.loc[mask, 'creation'] = 'included'
dataset_ids.loc[~mask, 'creation'] = 'new'
dataset_ids = dataset_ids.melt(id_vars=['dataset', 'creation'],
                               value_vars=['num_ids_std', 'num_ids_sus'],
                               var_name='search_mode', value_name='num_ids')

sns.boxplot(x='creation', y='num_ids', hue='search_mode', data=dataset_ids,
            dodge=True, ax=ax, meanline=True, showfliers=False,
            showmeans=True, meanprops={'color': '#2f2f2f'})
for patch in ax.artists:
    patch.set_facecolor((*patch.get_facecolor()[:3], .75))
np.random.seed(1)    # Control jitter.
sns.stripplot(x='creation', y='num_ids', hue='search_mode', data=dataset_ids,
              dodge=True, size=3, edgecolor='gray', linewidth=0.3, ax=ax,
              marker='.', clip_on=False, zorder=10)

ax.set_ylim(0, 1)
ax.yaxis.set_major_formatter(mticker.PercentFormatter(1, 0))

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:2],
          ['Standard libraries', 'Standard libraries +\nsuspect library'],
          loc='upper right')

n_included = len(msv_ids)
n_not_included = len(set(filename_ids['dataset'].unique()) - set(msv_ids))
ax.set_xticklabels(
    [f'Included during creation\n({n_included} datasets)',
     f'Not included during creation\n({n_not_included} datasets)'])
ax.set_xlabel('')
ax.set_ylabel('Spectrum match rate')

sns.despine(ax=ax)

plt.savefig('evaluate_annotations.png', dpi=300, bbox_inches='tight',
            facecolor='white')
plt.show()
plt.close()

In [ ]:
width = 7
height = width / 1.618
fig, ax = plt.subplots(figsize=(width, height))

dataset_ids = (filename_ids.groupby('dataset')
               .agg({'spectra_ms2': 'sum',
                     'num_ids_std': 'sum',
                     'num_ids_sus': 'sum'})
               .reset_index())
dataset_ids = dataset_ids[dataset_ids['num_ids_std'] > 0]
dataset_ids['num_ids_fold'] = (dataset_ids['num_ids_sus']
                               / dataset_ids['num_ids_std'])
mask = dataset_ids['dataset'].isin(msv_ids)
dataset_ids.loc[mask, 'creation'] = 'included'
dataset_ids.loc[~mask, 'creation'] = 'new'
dataset_ids = dataset_ids.melt(id_vars=['dataset', 'creation'],
                               value_vars=['num_ids_fold'],
                               var_name='search_mode', value_name='num_ids')

max_fold_change = 10
dataset_ids.loc[dataset_ids['num_ids'] > max_fold_change,
                'num_ids'] = max_fold_change

sns.histplot(x='num_ids', hue='creation', data=dataset_ids, stat='count',
             binwidth=0.5, multiple='layer', hue_order=['included', 'new'],
             legend=True, ax=ax)

ax.set_yscale('log')

ax.xaxis.set_major_locator(mticker.FixedLocator([1.75, 3.75, 5.75, 7.75, 9.75]))
ax.set_xticklabels(['2', '4', '6', '8', '≥10'])

ax.set_xlabel('Spectrum match rate fold change')
ax.set_ylabel('Number of datasets')

ax.legend(ax.get_legend().legendHandles,
          ['Included during creation', 'Not included during creation'],
          loc='upper right')

sns.despine(ax=ax)

plt.savefig('evaluate_annotations_fold_change.png', dpi=300,
            bbox_inches='tight', facecolor='white')
plt.show()
plt.close()